## Mixed Integer Linear Programing 
### E-Scooter Sharing Scheme
Conditional Service 

In [1]:
import pandas as pd
import numpy as np
import itertools
import os
import collections
import time
from pulp import *

In [2]:
minimise = True
reduced = False
# 8, 12, 15
demand_percent = 12
# 0, 1, 2
# 0 - low (1 year)
# 1 - default (2.5 years)
# 2 - high (5 years)
costs = 1
budget = 1

In [3]:
if minimise:
    problem = LpProblem("E-Scooter Allocation Carbon Minimisation", LpMinimize)
else:
    problem = LpProblem("E-Scooter Allocation Profit Maximisation", LpMaximize)

C:\Users\zoria\miniconda3\envs\py37\lib\site-packages\pulp\pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [4]:
if reduced:
    df_distance = pd.read_csv("Data/Preliminary analysis Data/Model Data - Smaller set Distance.csv")
else:
    df_distance = pd.read_csv("Data/Model Data - 56loc Distance.csv")

In [5]:
df_distance.fillna(0, inplace=True)
distance = df_distance.values[:,1:]
distance_dict = { (x,y): distance[x][y] for x in range(distance.shape[0]) for y in range(distance.shape[1])}

In [6]:
locations = list(df_distance['Location'])
location_idx = np.arange(0, len(locations))
loc_count = len(locations)

In [7]:
if reduced:
    files = os.listdir('Data/Preliminary analysis Data/Reduced set Demand/')
else:
    files = os.listdir('Data/Final set Demand/')

In [8]:
d = dict()
for f in files:
    hour = int(f.split(sep='.', maxsplit=1)[0])
    if reduced:
        df = pd.read_csv('Data/Preliminary analysis Data/Reduced set Demand/' + f)
    else:
        df = pd.read_csv('Data/Final set Demand/' + f)
    demand_arr = df.to_numpy()[:,1:]
    demand_arr = demand_arr*(demand_percent/15)
    for i in range(loc_count):
        for j in range(loc_count):
            demand_arr[i][j] = np.round(demand_arr[i][j],0)
    d[hour] = demand_arr

In [9]:
demand = collections.OrderedDict(sorted(d.items())) 

In [10]:
demand_dict = dict()
for i,(_,v) in enumerate(demand.items()):
    demand_dict.update({(x,y,i): v[x][y] for x in range(loc_count) for y in range(loc_count)})

In [11]:
# Parameters
M = sys.maxsize
Zmax = 100
Zmin = 1
if costs == 0:
    # 1 year
    Cc_fixed_scooter = 140
    Cc_fixed_dock = 74
    Cc_fixed_station = 95

    Cp_fixed_scooter = 0.96
    Cp_fixed_dock = 2.3
    Cp_fixed_station = 5.75
elif costs == 1:
    # 2.5 years
    Cc_fixed_scooter = 55
    Cc_fixed_dock = 30
    Cc_fixed_station = 38

    Cp_fixed_scooter = 0.38
    Cp_fixed_dock = 0.92
    Cp_fixed_station = 2.3  
else:
    # 5 years
    Cc_fixed_scooter = 31
    Cc_fixed_dock = 15
    Cc_fixed_station = 19

    Cp_fixed_scooter = 0.19
    Cp_fixed_dock = 0.46
    Cp_fixed_station = 1.15

# Carbon Cost values
Cc_scooter_km = 7
# Cost relocation
Cc_r = 120

scalar = 1.25

# Cost values
# cost of maintaining one scooter per kilometer driven
Cp_scooter_km = 0.41

# Cost relocation
Cp_r = 1.5
# Price rate for kilometer driven
P_km = 0.6
# Price for pickup
P_init = 1
if reduced:
    B = 500
else:
    if budget == 0:
        B = 900
    elif budget == 1:
        B = 1200
    else:
        B = 1500

In [12]:
# Penalty
if reduced:
    df_penalty = pd.read_csv("Data/Preliminary analysis Data/Penalty Carbon Costs Smaller set.csv")
else:
    df_penalty = pd.read_csv("Data/Penalty Carbon Costs 56loc.csv")
C_pen = df_penalty.values[:, 1]
# C_pen = C_pen*scalar

In [13]:
# Sets
T = np.arange(0,len(files)+1)
# each location at a given time
X = list(itertools.product(location_idx, T))
X2 = [(i,it) for (i,it) in X if sum([demand_dict[(i,j,it)] for j in location_idx if i !=j and it != T[-1]]) > 0]
A1 = [(xi, xj) for xi in X for xj in X if xi[0] != xj[0] and xi[1]+1==xj[1]]
# Relocation
A2 = [(xi, xj) for xi in X for xj in X if xi[0]!=xj[0] and xi[1]==T[-1] and xj[1]==T[0]]

In [14]:
# Decision Variables
Yi = LpVariable.dicts("Station Presence", location_idx,0,cat=const.LpBinary)
Zi = LpVariable.dicts("Size", location_idx, 0, cat=const.LpInteger)
# Relocation
Rij = LpVariable.dicts("#Relocated_Scooters", A2, 0, cat=const.LpInteger)
R = LpVariable.dicts('Relocation needed', A2, 0, cat=const.LpBinary)
Vit = LpVariable.dicts("#Available_Scooters",X,0,cat=const.LpInteger)
Ditj = LpVariable.dicts("#Used_Scooters", A1, 0 ,cat=const.LpInteger)
Xit = LpVariable.dicts("Availability binary", X2, 0, cat=const.LpBinary)

In [15]:
# Objective function
if minimise:
    problem+=lpSum(Ditj[((i,ti),(j,tj))]*distance_dict[(i,j)]*Cc_scooter_km for ((i,ti),(j,tj)) in A1) + \
    lpSum((demand_dict[(i,j,ti)]-Ditj[((i,ti),(j,tj))])*distance_dict[(i,j)]*C_pen[i] for ((i,ti),(j,tj)) in A1) + \
    lpSum(R[((i,ti), (j,tj))]*distance_dict[(i,j)]*Cc_r for ((i,ti), (j,tj)) in A2) + \
    lpSum(Vit[(i,t)]*Cc_fixed_scooter for (i,t) in X if t==0) + \
    lpSum(Zi[i]*Cc_fixed_dock for i in location_idx) + \
    lpSum(Yi[i]*Cc_fixed_station for i in location_idx), "Objective function"

else:
    problem+=lpSum(Ditj[((i,ti),(j,tj))]*(distance_dict[(i,j)]*(P_km-Cp_scooter_km) + P_init) for ((i,ti),(j,tj)) in A1) - \
    lpSum(R[((i,ti), (j,tj))]*distance_dict[(i,j)]*Cp_r for ((i,ti), (j,tj)) in A2) - \
    lpSum(Vit[(i,t)]*Cp_fixed_scooter for (i,t) in X if t==0) - \
    lpSum(Zi[i]*Cp_fixed_dock for i in location_idx) - \
    lpSum(Yi[i]*Cp_fixed_station for i in location_idx), "Objective function"


In [16]:
for (i,ti) in X:
    if ti !=0:
        problem+= Vit[(i,ti-1)] - lpSum(Ditj[((i,ti-1),(j,ti))] for j in location_idx if i !=j) + lpSum(Ditj[((j, ti-1),(i, ti))] for j in location_idx if i !=j) == Vit[(i,ti)], f"Availability Balance {(i,ti)}"


In [17]:
for (i,ti) in X:
    if ti!=T[-1]:
        problem+=Vit[(i,ti)] - lpSum(Ditj[((i,ti),(j,ti+1))] for j in location_idx if i !=j) >=0, f"Stocked Scooters for {(i,ti)}"

In [18]:
# Relocation constraint
for i in location_idx:
    problem+=Vit[(i,0)] == Vit[(i,T[-1])] + lpSum(Rij[((j,T[-1]), (i,0))] for j in location_idx if i!=j) - lpSum(Rij[((i,T[-1]), (j,0))] for j in location_idx if i!=j), f"Relocation balance for location {i}"

In [19]:
for a2 in A2:
    problem+=Rij[a2]<=M*R[a2], f"Relocation needed for {a2}"

In [20]:
for i in location_idx:
    problem+=lpSum(Rij[((i,T[-1]), (j,0))] for j in location_idx if i!=j) <= Vit[(i,T[-1])], f"Relocation availability for location {i}"

In [21]:
for (i,ti) in X:
    problem+= Zi[i] >=Vit[(i,ti)], f"Size constraint for {(i,ti)}"

In [22]:
for (i,ti) in X:
    problem+=Vit[(i,ti)] <= Zmax*Yi[i], f"Maximum size constraint {(i,ti)}"

In [23]:
for i in location_idx:
    problem+=Yi[i]*Zmin <=Zi[i], f"Minimum size constraint for {i}"

In [24]:
for ((i,ti),(j,tj)) in A1:
    problem+=Ditj[((i,ti),(j,tj))]<=demand_dict[(i,j,ti)], f"Maximum Demand for {((i,ti),(j,tj))}"

In [25]:
problem+=lpSum(Vit[(i,0)]*Cp_fixed_scooter for i in location_idx) + lpSum(Zi[i]*Cp_fixed_dock for i in location_idx) + lpSum(Yi[i]*Cp_fixed_station for i in location_idx)<=B, "Maximum Budget"

In [26]:
for (i,ti) in X2:
    problem+= Xit[(i,ti)] <= (Vit[(i,ti)] + lpSum(demand_dict[(i,j,ti)] for j in location_idx if i!=j) - lpSum(demand_dict[(i,j,ti)]*Yi[j] for j in location_idx if i!=j))/lpSum(demand_dict[(i,j,ti)] for j in location_idx if i!=j), f"Xit value constraint for {(i,ti)}"

In [27]:
for (i,ti) in X2:
    if ti != T[-1]:
        problem+=Vit[(i,ti)] - lpSum(Ditj[((i,ti),(j,ti+1))] for j in location_idx if i !=j) <= M*Xit[(i,ti)]

In [28]:
for (i,ti) in X2:
    if ti != T[-1]:
        problem+= lpSum(Ditj[((i,ti), (j,ti+1))] for j in location_idx if i!=j) >= M*(Xit[(i,ti)]-1) + lpSum(demand_dict[(i,j,ti)]*Yi[j] for j in location_idx if i!=j)

In [29]:
if minimise:
    if reduced:
        logf = "stats_carbon_reduced_NOcuts.log"
        problem.writeLP('Models/carbon_reduced_model_NOcuts.lp')
    else:
        logf = "stats_carbon_final_NOcuts.log"
        problem.writeLP('Models/carbon_model_NOcuts.lp')
else:
    if reduced:
        logf = "stats_profit_reduced.log"
        problem.writeLP('Models/profit_reduced_model.lp')
    else:
        logf = "stats_profit_final_demand8.log"
        problem.writeLP('Models/profit_model.lp')


In [30]:
solver = apis.PULP_CBC_CMD(logPath=logf, options=['DivingVectorlength on', 'DivingSome on', '-cuts off'], gapRel=0.01, timeLimit = 60*60*3)
start = time.time()
solver.actualSolve(problem)
end = time.time()
print(f'Running time: {str(int(end-start))}s')
LpStatus[problem.status]

C:\Users\zoria\miniconda3\envs\py37\lib\site-packages\pulp\apis\coin_api.py:146: UserWarning: `logPath` argument replaces `msg=1`. The output will be redirected to the log file.
  warnings.warn('`logPath` argument replaces `msg=1`. The output will be redirected to the log file.')
Running time: 10804s


'Optimal'

In [31]:
obj_val_carbon = sum([Ditj[((i,ti),(j,tj))].value()*distance_dict[(i,j)]*Cc_scooter_km for ((i,ti),(j,tj)) in A1]) + \
    sum([(demand_dict[(i,j,ti)]-Ditj[((i,ti),(j,tj))].value())*distance_dict[(i,j)]*C_pen[i] for ((i,ti),(j,tj)) in A1]) + \
    sum([R[((i,ti), (j,tj))].value()*distance_dict[(i,j)]*Cc_r for ((i,ti), (j,tj)) in A2]) + \
    sum([Vit[(i,t)].value()*Cc_fixed_scooter for (i,t) in X if t==0]) + \
    sum([Zi[i].value()*Cc_fixed_dock for i in location_idx]) + \
    sum([Yi[i].value()*Cc_fixed_station for i in location_idx])
print(f"Objective value carbon: {obj_val_carbon}")

Objective value carbon: 390110.5771088993


In [32]:
obj_val_profit = sum([Ditj[((i,ti),(j,tj))].value()*(distance_dict[(i,j)]*(P_km-Cp_scooter_km) + P_init) for ((i,ti),(j,tj)) in A1]) - \
    sum([R[((i,ti), (j,tj))].value()*distance_dict[(i,j)]*Cp_r for ((i,ti), (j,tj)) in A2]) - \
    sum([Vit[(i,t)].value()*Cp_fixed_scooter for (i,t) in X if t==0]) - \
    sum([Zi[i].value()*Cp_fixed_dock for i in location_idx]) - \
    sum([Yi[i].value()*Cp_fixed_station for i in location_idx])
print(f"Objective value profit: {obj_val_profit}")

Objective value profit: 3133.869369999991


In [33]:
value(problem.objective)

390110.5771089031

In [34]:
# Extracting the results
#(idx, LpVariable)
size_list = []
for _,v in Zi.items():
    size_list.append(v.value())  

In [35]:
station_list = []
for _,v in Yi.items():
    station_list.append(v.value()) 

In [36]:
df_results = pd.DataFrame(list(zip(locations, station_list, size_list)), columns=["Location","Station", "Size"])
df_results

,Location,Station,Size
0,Abbeyhill,1.0,2.0
1,Balgreen and Roseburn,1.0,17.0
2,"Blackford, West Mains and Mayfield Road",1.0,17.0
3,Bonnington,1.0,10.0
4,Boswall and Pilton,1.0,31.0
5,Broughton North and Powderhall,1.0,17.0
6,Broughton South,1.0,18.0
7,Bruntsfield,1.0,10.0
8,"Canongate, Southside and Dumbiedykes",1.0,16.0
9,Canonmills and New Town North,1.0,7.0


In [37]:
if minimise:
    if reduced:
        df_results.to_csv("results-carbon-reduced-NOcuts.csv", index=False)
    else:
        df_results.to_csv("results-carbon-NOcuts.csv", index=False)
else:
    if reduced:
        df_results.to_csv("results-profit-reduced.csv", index=False)
    else:
        df_results.to_csv("results-profit_demand8.csv", index=False)

In [38]:
total_Vit = dict()
for k,v in Vit.items():
    if k[1] not in total_Vit:
        total_Vit[k[1]] = int(v.varValue)
    else:
        total_Vit[k[1]]+=int(v.varValue)

In [39]:
total_scooters = total_Vit[0]
print("#Scooters: " + str(total_Vit[0]))

#Scooters: 512


In [40]:
total_demand = 0
for _,v in demand_dict.items():
    total_demand+=v

satisfied_demand = 0
for _,v in Ditj.items():
    satisfied_demand+=v.varValue
satisfied_demand

print(f'Satisfied demand: {satisfied_demand/total_demand}')
print(f'Avg. Scooter rides per day: {satisfied_demand/total_scooters}')
print(f'Trips made: {satisfied_demand}')
print(f'Total number of trips: {total_demand}')

Satisfied demand: 0.5277866496327052
Avg. Scooter rides per day: 6.455078125
Trips made: 3305.0
Total number of trips: 6262.0


In [41]:
satisfied_demand

3305.0

In [42]:
trips = dict()
for ((i,ti),(j,tj)) in Ditj.keys():
    if ti+5 not in trips.keys():
        trips[ti+5] = int(Ditj[((i,ti),(j,tj))].varValue)
    else:
        trips[ti+5] +=int(Ditj[((i,ti),(j,tj))].varValue)
trips

{5: 119,
 6: 318,
 7: 505,
 8: 336,
 9: 103,
 10: 28,
 11: 8,
 12: 9,
 13: 8,
 14: 171,
 15: 385,
 16: 390,
 17: 458,
 18: 294,
 19: 107,
 20: 35,
 21: 9,
 22: 10,
 23: 12}

In [43]:
demand_t = dict()
for (i,j,t), v in demand_dict.items():
    if t+5 not in demand_t.keys():
        demand_t[t+5] = int(demand_dict[(i,j,t)])
    else:
        demand_t[t+5] +=int(demand_dict[(i,j,t)])
demand_t

{5: 241,
 6: 630,
 7: 1039,
 8: 548,
 9: 170,
 10: 64,
 11: 15,
 12: 26,
 13: 26,
 14: 270,
 15: 673,
 16: 673,
 17: 1039,
 18: 548,
 19: 169,
 20: 64,
 21: 15,
 22: 26,
 23: 26}

In [44]:
satisfied_demand_t = dict()
for t in demand_t.keys():
    satisfied_demand_t[t] = trips[t] / demand_t[t]
satisfied_demand_t

{5: 0.49377593360995853,
 6: 0.5047619047619047,
 7: 0.48604427333974976,
 8: 0.6131386861313869,
 9: 0.6058823529411764,
 10: 0.4375,
 11: 0.5333333333333333,
 12: 0.34615384615384615,
 13: 0.3076923076923077,
 14: 0.6333333333333333,
 15: 0.5720653789004457,
 16: 0.5794947994056464,
 17: 0.4408084696823869,
 18: 0.5364963503649635,
 19: 0.6331360946745562,
 20: 0.546875,
 21: 0.6,
 22: 0.38461538461538464,
 23: 0.46153846153846156}

In [45]:
# df_satisfied_demand = pd.DataFrame(satisfied_demand_t.items())
# df_satisfied_demand.to_csv("proportion of satisfied demand per timestep profit.csv", index=False)

In [46]:
for k,v in Rij.items():
    if v.varValue>0:
        print(f"Rij[{k}] = {v.varValue}")

Rij[((8, 19), (29, 0))] = 2.0
Rij[((10, 19), (11, 0))] = 3.0
Rij[((34, 19), (50, 0))] = 2.0
Rij[((35, 19), (5, 0))] = 2.0
Rij[((36, 19), (2, 0))] = 2.0
Rij[((49, 19), (54, 0))] = 1.0


In [47]:
fixed_cost = sum([Vit[(i,0)].value()*Cp_fixed_scooter for i in location_idx]) + sum([Zi[i].value()*Cp_fixed_dock for i in location_idx]) + sum([Yi[i].value()*Cp_fixed_station for i in location_idx])
print(f"Total fixed cost: {fixed_cost}")

Total fixed cost: 1199.6599999999999


In [48]:
loc_trips = dict()

for l in location_idx:
    loc = locations[l]
    df = pd.DataFrame(columns=['Time', 'Available', 'Leaving', 'Arriving', 'Demand', 'Xit'])
    for t in T:
        if t!=T[-1]:
            available = Vit[(l,t)].value()
            leaving = sum([Ditj[((l,t),(j,t+1))].value() for j in location_idx if j!=l and t!=T[-1]])
            arriving = sum([Ditj[((j,t),(l,t+1))].value() for j in location_idx if j!=l and t!=T[-1]])
            time = f"{int(t+5)}:00-{int(t+5)}:59"
            d = sum([demand_dict[(l,j,t)] for j in location_idx if j!=l])
            x = Xit[(l,t)].value() if (l,t) in X2 else None
            df.loc[-1] = [time, int(available), int(leaving), int(arriving), int(d), x]
            df.index = df.index + 1
    relocate_from = sum([Rij[((l,T[-1]), (j,0))].value() for j in location_idx if j!=l])
    relocate_to = sum([Rij[((j,T[-1]), (l,0))].value() for j in location_idx if j!=l])
    df.loc[-1] = ['Relocate', None, int(relocate_from), int(relocate_to), None, None]
    df.index = df.index + 1
    loc_trips[loc] = df

In [49]:
import report_generator as r
r.station_stats(loc_trips)

C:\Users\zoria\miniconda3\envs\py37\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [50]:
import csv
variable_values = dict()
for v in problem.variables():
    variable_values[v.name] = v.value()

variable_values_df = pd.DataFrame.from_dict(variable_values, orient='index')
df_csv = variable_values_df.to_csv(index=True)
if minimise:
    if reduced:
        filename = 'variable_values_carbon_reduced_NOcuts.csv'
    else:
        filename = 'variable_values_carbon_NOcuts.csv'
else:
    if reduced:
        filename = 'variable_values_profit_reduced_demand8.csv'
    else:
        filename = 'variable_values_profit_demand8.csv'
 
with open(filename, 'w') as f:
    f.write(df_csv)